# Demo App
The demo app lets you easily run the model from you phone. Click the camera button below to take and analyze a picture. You can change the model by uploading a model to the notebooks directory

In [ ]:
import warnings # tf needs to learn to stfu
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [ ]:
from ipywebrtc import CameraStream, ImageRecorder # for capturing pictures
from IPython.display import display, Javascript
from keras.models import load_model # for loading models
from keras.applications.mobilenet_v2 import preprocess_input
import PIL
import numpy as np
import io
from pathlib import Path
from functools import lru_cache
import ipywidgets as ipw

@lru_cache()
def get_model(in_path):
    return load_model(in_path, compile=False)

In [ ]:
dx_name_dict = {
    'nv': 'melanocytic nevi',
    'mel': 'melanoma',
    'bcc': 'basal cell carcinoma',
    'akiec': 'Actinic keratoses and intraepithelial carcinoma',
    'vasc': 'vascular lesions',
    'bkl': 'benign keratosis-like',
    'df': 'dermatofibroma'
}
dx_name_id_dict = {id: name for id, name in enumerate(dx_name_dict.values())}
dx_name_vec = np.array([dx_name_id_dict[k] for k in sorted(dx_name_id_dict.keys())])

In [ ]:
empty_image_io = io.BytesIO()
PIL.Image.fromarray(np.zeros((3, 3), dtype='uint8')).save(empty_image_io, format='png')
def setup_appmode():
    js_str = """$('#appmode-leave').hide()
        // Hides the edit app button.
        $('#appmode-busy').hide()
        // Hides the kernel busy indicator.
        IPython.OutputArea.prototype._should_scroll = function(lines) {
            return false
            // disable scrolling
        }"""
    display(Javascript(js_str))

def get_app_user_id():
    """
    Get the userid from the `jupyter_notebook_url`
    injected by the appmode extension (if in use)
    otherwise return a 'nobody'
    :return: appmode username of current user
    >>> get_app_user_id()
    'nobody'
    >>> jupyter_notebook_url = 'https://a.b.c?user=dan#hello'
    >>> get_app_user_id()
    'dan'
    """
    cur_url = globals().get('jupyter_notebook_url', None)
    if cur_url is None:
        # black magic to get the 'injected' variable
        frame = inspect.currentframe()
        try:
            out_locals = frame.f_back.f_locals
            cur_url = out_locals.get('jupyter_notebook_url', 'nobody')
        finally:
            del frame
    qs_info = parse_qs(urlparse(cur_url).query)
    return qs_info.get('user', ['nobody'])[0]

In [ ]:
camera = CameraStream(constraints=
                      {'facing_mode': 'environment',
                       'audio': False,
                       'video': { 'width': 224, 'height': 224 }
                       })
image_recorder = ImageRecorder(stream=camera)

In [ ]:
output = ipw.Output()
progress_bar = ipw.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Waiting...',
    bar_style='info',
    orientation='horizontal'
)
status_text = ipw.Text(description='Status', value='')

# camera selection code
camera_options = [('Back-camera', 'environment'), ('Selfie', 'user')]
camera_sel = ipw.ToggleButtons(options=camera_options)


def _camera_switch(change):
    camera.constraints['facing_mode'] = camera_sel.value


camera_sel.observe(_camera_switch, 'value')

# model selection code
model_list = [(c_model.stem.replace('_', ' '), c_model) for c_model in
              Path('..').glob('**/*.h5')]
model_sel = ipw.ToggleButtons(options=model_list)


def _model_switch(change):
    progress_bar.value = 0
    progress_bar.bar_style = 'info'
    get_model(model_sel.value)
    update_image(change)


model_sel.observe(_model_switch, 'value')


# image processing code

@output.capture()
def update_image(change):
    progress_bar.value = 0
    progress_bar.bar_style = 'info'
    status_text.value = 'Loading...'
    if len(image_recorder.image.value) < 1:
        progress_bar.description = 'No Image'
        progress_bar.value = 10
        progress_bar.bar_style = 'danger'
        c_image_io = empty_image_io
    else:
        c_image_io = io.BytesIO(image_recorder.image.value)

    single_img = PIL.Image.open(c_image_io)
    progress_bar.value = 10
    progress_bar.description = 'Fetching Model...'
    c_model = get_model(model_sel.value)
    progress_bar.description = 'Processing Image...'
    progress_bar.value = 30

    model_shape = [x if x is not None else 224
                   for x in c_model.get_input_shape_at(0)[1:3]]
    clean_img = single_img.convert('RGB').resize(model_shape)
    single_img_arr = np.expand_dims(np.array(clean_img), 0)
    nn_input_arr = preprocess_input(single_img_arr)
    progress_bar.value = 50
    progress_bar.description = 'Running model...'
    nn_output_arr = c_model.predict(nn_input_arr)
    progress_bar.value = 90
    status_text.value = f'{dx_name_vec[np.argmax(nn_output_arr)]}, {np.max(nn_output_arr):2.1%}'
    progress_bar.value = 100
    progress_bar.bar_style = 'success'


sel_items = ipw.VBox([
    ipw.HBox([ipw.Label(value='Select Camera'), camera_sel]),
    ipw.HBox([ipw.Label(value='Select Model'), model_sel])
])

image_recorder.image.observe(update_image, 'value')
sr_box = ipw.VBox()
_model_switch(0)  # preload everything
ipw.VBox([
    sel_items,
    ipw.HBox([camera,
              sr_box
              ]),
    image_recorder,
    progress_bar,
    status_text,
    output
])